In [4]:
import pandas as pd
import requests
from shapely.geometry import Point
import geopandas as gpd

In [26]:
# Leer el archivo y procesar las coordenadas
df = pd.read_csv('../data/bicimad_stations.csv', sep="\t")
df[['longitud', 'latitud']] = df['geometry.coordinates'].str[1:-1].str.split(', ', expand=True).astype(float)

# Extraer el nombre de la estación y renombrar columnas
df['name'] = df['name'].str.extract(r'\s-\s(.*)')
bicimad_stations = df[['name', 'address', 'latitud', 'longitud', 'dock_bikes']].rename(columns={
    'name': 'BiciMAD station', 
    'address': 'Station location', 
    'latitud': 'station_lat',
    'longitud': 'station_long',
    'dock_bikes': 'Bikes_available' 
})

#bicimad_stations

In [27]:
# Acceder a la API
url = 'https://datos.madrid.es/egob/catalogo/300614-0-centros-educativos.json'
response = requests.get(url)

# Convertir la respuesta a JSON
json_data = response.json()

# Crear DataFrame con la información de interés
df = pd.DataFrame(json_data['@graph'])

# Extraer la dirección, latitud y longitud
df['street_address'] = df['address'].apply(lambda x: x.get('street-address') if pd.notnull(x) else None).str.title()
df['latitude'] = df['location'].apply(lambda x: x['latitude'] if isinstance(x, dict) else None)
df['longitude'] = df['location'].apply(lambda x: x['longitude'] if isinstance(x, dict) else None)

# Crear el DataFrame final con las columnas relevantes
df_places = df[['title', 'street_address', 'latitude', 'longitude']].rename(columns={
    'title': 'Place of interest', 'street_address': 'Place address', 'latitude': 'place_lat', 'longitude': 'place_long'})

# Eliminar filas con valores nulos y restablecer el índice
df_places = df_places.dropna().reset_index(drop=True)


#df_places

In [28]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [29]:
# Utilizamos los DataFrames de prueba: places_prueba y stations_prueba
places_prueba = bicimad_stations.head(50)  
stations_prueba = df_places.head(50)      

In [30]:
#DF final con valores de prueba
df_combined = pd.merge(stations_prueba, places_prueba, how='cross') 

df_combined['distance'] = df_combined.apply(lambda row: distance_meters(
    row['station_lat'], row['station_long'], row['place_lat'], row['place_long']).item(), axis=1)

nearest_stations = df_combined.loc[df_combined.groupby('Place of interest')['distance'].idxmin()]

final_result = nearest_stations[['Place of interest', 'Place address', 'BiciMAD station', 'Station location', 'Bikes_available']]
final_result

,Place of interest,Place address,BiciMAD station,Station location,Bikes_available
8,ABC Schule (Alemán),Calle Leizaran 27,Alonso Martínez,Plaza de Alonso Martínez nº 5,22
110,AFI Escuela de Finanzas,Calle Marques De Villamejor 5,Marqués de la Ensenada,Calle Marqués de la Ensenada nº 16,8
66,Academia Darío Estudio,Calle Gran Via 63,San Bernardo,Calle San Bernardo nº 22,17
158,Agencia Estatal Consejo Superior de Investigac...,Calle Serrano 117,Alonso Martínez,Plaza de Alonso Martínez nº 5,22
208,"Altair, Colegio Internacional",Calle Joaquin Bau 4,Alonso Martínez,Plaza de Alonso Martínez nº 5,22
346,Aula Municipal de Formación de Personas Adulta...,Calle Casino 5,Ribera de Curtidores,Calle Ribera de Curtidores nº 28,9
358,Aula Municipal de Formación de Personas Adulta...,Calle Almagro 5 I.F.E. Del Gobierno Local De M...,Alonso Martínez,Plaza de Alonso Martínez nº 5,22
410,Aula Municipal de Formación de Personas Adulta...,Calle Santa Felicidad 39 En Centro Cultural Elipa,Marqués de la Ensenada,Calle Marqués de la Ensenada nº 16,8
460,Aula Municipal de Formación de Personas Adulta...,Calle San Roman Del Valle 8 En Centro Cultural...,Marqués de la Ensenada,Calle Marqués de la Ensenada nº 16,8
547,Aula Municipal de Formación de Personas Adulta...,Calle Doctor Martin Arevalo 2 Centro Cultural ...,Embajadores 1,Glorieta de Embajadores nº 6,10


In [15]:
#Guardar CSV
final_result.to_csv("nearest_bicimad_stations.csv", index=False)